In [18]:
import pandas as pd
import numpy as np
df=pd.read_excel('top100cp.xlsx');#df.head(2)

In [19]:
df['Date'] = pd.to_datetime(df['Date'])  
df.set_index('Date', inplace=True);df.head(3)

,A B B India Ltd.,Adani Energy Solutions Ltd.,Adani Enterprises Ltd.,Adani Green Energy Ltd.,Adani Ports & Special Economic Zone Ltd.,Adani Total Gas Ltd.,Adani Wilmar Ltd.,Ambuja Cements Ltd.,Apollo Hospitals Enterprise Ltd.,Asian Paints Ltd.,...,Torrent Pharmaceuticals Ltd.,Trent Ltd.,U P L Ltd.,Ultratech Cement Ltd.,United Spirits Ltd.,Varun Beverages Ltd.,Vedanta Ltd.,Wipro Ltd.,Zomato Ltd.,Zydus Lifesciences Ltd.
Date,,,,,,,,,,,,,,,,,,,,,
2023-04-03,3373.75,943.40,1717.45,837.45,627.80,845.35,395.70,374.80,4234.35,2777.00,...,1545.30,1357.40,729.65,7640.50,750.65,1414.45,282.40,367.95,51.95,492.40
2023-04-05,3374.15,907.85,1698.00,815.60,636.75,821.95,396.95,379.95,4200.95,2808.50,...,1578.10,1364.80,732.30,7691.95,755.55,1454.00,286.25,369.75,51.70,493.10
2023-04-06,3397.10,953.20,1754.05,856.35,641.60,863.00,409.20,383.60,4201.95,2809.85,...,1574.75,1367.25,726.15,7701.85,756.45,1393.50,273.20,366.65,52.15,494.15


In [20]:
#under the equilibrium assumption, the tangency portfolio acts as the market portfolio.
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
mu = mean_historical_return(df)
cov_matrix = CovarianceShrinkage(df).ledoit_wolf()
inv_covariance_matrix = np.linalg.inv(cov_matrix)


from pypfopt.efficient_frontier import EfficientFrontier
ef = EfficientFrontier(mu, cov_matrix)
weights = ef.max_sharpe()
wt = pd.DataFrame(weights.items(), columns=['asset', 'weight'])
wt.set_index('asset', inplace=True);wt

portfolio_return, portfolio_variance, _ = ef.portfolio_performance()

In [21]:
wt=wt.reset_index();wt.shape;print(wt)

                                       asset    weight
0                           A B B India Ltd.  0.000003
1                Adani Energy Solutions Ltd.  0.000029
2                     Adani Enterprises Ltd.  0.000018
3                    Adani Green Energy Ltd.  0.000020
4   Adani Ports & Special Economic Zone Ltd.  0.000011
..                                       ...       ...
95                      Varun Beverages Ltd. -0.000002
96                              Vedanta Ltd.  0.000005
97                                Wipro Ltd.  0.000002
98                               Zomato Ltd.  0.175261
99                   Zydus Lifesciences Ltd.  0.079432

[100 rows x 2 columns]


# Index weights and daily prices

In [22]:
wtt=np.array(wt.weight).reshape(100,1);print(wtt.shape)
st=np.array(df.values);print(st,st.shape)
market=st@wtt;print('market returns is:',market,market.shape) #this is our index for the 100 selected asset

(100, 1)
[[3373.75  943.4  1717.45 ...  367.95   51.95  492.4 ]
 [3374.15  907.85 1698.   ...  369.75   51.7   493.1 ]
 [3397.1   953.2  1754.05 ...  366.65   52.15  494.15]
 ...
 [5864.9  1008.9  3066.5  ...  500.45  169.65  993.15]
 [5967.4  1015.6  3107.7  ...  487.1   174.2   999.75]
 [5929.8  1045.9  3109.95 ...  479.85  182.6  1017.05]] (244, 100)
market returns is: [[1107.96325101]
 [1115.72156056]
 [1119.11613031]
 [1125.92147603]
 [1139.71609769]
 [1157.33879496]
 [1157.81158329]
 [1156.40434749]
 [1152.55098457]
 [1157.25758022]
 [1168.02787008]
 [1164.48385988]
 [1169.59994756]
 [1171.45494388]
 [1166.98784491]
 [1183.53527829]
 [1187.72102236]
 [1201.60212533]
 [1196.29921202]
 [1201.52142332]
 [1203.76202947]
 [1221.31620993]
 [1221.37970135]
 [1228.81089051]
 [1233.81869004]
 [1235.32308308]
 [1248.16640159]
 [1241.9432601 ]
 [1242.42900197]
 [1233.81855417]
 [1229.5759648 ]
 [1237.12998425]
 [1235.44253883]
 [1237.42803592]
 [1257.23728283]
 [1260.0035901 ]
 [1265.551841

In [23]:
tangent_weights = wt.weight;tangent_weights

0     0.000003
1     0.000029
2     0.000018
3     0.000020
4     0.000011
        ...   
95   -0.000002
96    0.000005
97    0.000002
98    0.175261
99    0.079432
Name: weight, Length: 100, dtype: float64

In [24]:
t_weights = np.array(tangent_weights).reshape(-1,1);print('tangent weights shape:',t_weights.shape)
m_vec = np.array(mu).reshape(-1,1);print('mean vector shape:',m_vec.shape)
market_ret = np.transpose(t_weights)@m_vec;print('market rate of return is:',market_ret)
market_var = portfolio_variance;print('market variance is:',portfolio_variance)

tangent weights shape: (100, 1)
mean vector shape: (100, 1)
market rate of return is: [[1.55700005]]
market variance is: 0.12187634763843419


# Individual asset returns

In [25]:
market=pd.DataFrame(market,columns=['daily price'])
market['returns']=market.pct_change()
print(market.head(3))

mr=np.array(market.returns).reshape(244,1);print(mr)

   daily price   returns
0  1107.963251       NaN
1  1115.721561  0.007002
2  1119.116130  0.003042
[[            nan]
 [ 7.00231668e-03]
 [ 3.04248826e-03]
 [ 6.08100048e-03]
 [ 1.22518506e-02]
 [ 1.54623571e-02]
 [ 4.08513331e-04]
 [-1.21542730e-03]
 [-3.33219338e-03]
 [ 4.08363336e-03]
 [ 9.30673520e-03]
 [-3.03418291e-03]
 [ 4.39343804e-03]
 [ 1.58600923e-03]
 [-3.81329132e-03]
 [ 1.41796107e-02]
 [ 3.53664495e-03]
 [ 1.16871746e-02]
 [-4.41320234e-03]
 [ 4.36530531e-03]
 [ 1.86480749e-03]
 [ 1.45827664e-02]
 [ 5.19860623e-05]
 [ 6.08425796e-03]
 [ 4.07532157e-03]
 [ 1.21929831e-03]
 [ 1.03967283e-02]
 [-4.98582680e-03]
 [ 3.91114382e-04]
 [-6.93033388e-03]
 [-3.43858451e-03]
 [ 6.14359720e-03]
 [-1.36400010e-03]
 [ 1.60711407e-03]
 [ 1.60084032e-02]
 [ 2.20030642e-03]
 [ 4.40336153e-03]
 [-7.76871803e-04]
 [-4.82040652e-04]
 [ 9.71030268e-03]
 [ 1.11855466e-02]
 [ 4.60621565e-03]
 [ 5.45475986e-03]
 [ 8.30698177e-03]
 [-1.77315181e-03]
 [-5.39988200e-03]
 [ 2.52649549e-03]
 [ 1.11

In [26]:
assett=df['Bajaj Auto Ltd.'].copy()
assett['returns']=assett.pct_change()
#print(assett.returns.values)
a=np.array(assett.returns.values).reshape(244,1);print(a)

[[        nan]
 [ 0.0073615 ]
 [ 0.00259747]
 [ 0.00694169]
 [ 0.02851093]
 [ 0.02599703]
 [ 0.0005483 ]
 [-0.00792846]
 [-0.01060091]
 [ 0.0134347 ]
 [ 0.01138122]
 [-0.0014023 ]
 [ 0.00552422]
 [ 0.00241223]
 [-0.01002867]
 [ 0.02540126]
 [ 0.00538769]
 [ 0.01490315]
 [-0.0111494 ]
 [ 0.00221455]
 [ 0.00121139]
 [ 0.01930273]
 [-0.00448426]
 [ 0.00598386]
 [-0.00176692]
 [-0.00197894]
 [ 0.00565115]
 [-0.00573989]
 [-0.0019831 ]
 [-0.00236237]
 [-0.00784526]
 [ 0.00893335]
 [-0.00556016]
 [ 0.00433516]
 [ 0.02797946]
 [-0.00604005]
 [ 0.00055243]
 [-0.00575945]
 [-0.00538993]
 [ 0.0167391 ]
 [ 0.00549143]
 [ 0.00877043]
 [ 0.00343945]
 [ 0.01153134]
 [ 0.00416253]
 [-0.01192547]
 [ 0.00553401]
 [-0.00287233]
 [-0.00541427]
 [-0.00182868]
 [-0.01685887]
 [-0.00960803]
 [ 0.01406245]
 [-0.00254183]
 [-0.00818253]
 [ 0.00227662]
 [-0.00339636]
 [ 0.00242028]
 [ 0.02127521]
 [-0.00524776]
 [-0.01724377]
 [ 0.00319912]
 [ 0.05721667]
 [ 0.00565434]
 [-0.01702015]
 [ 0.01322921]
 [ 0.00402

In [27]:
#main=pd.DataFrame(mr,a,columns=['asset ret','market ret']);main.head()
main = pd.DataFrame(data=np.concatenate([mr, a], axis=1), columns=['market_ret', 'asset_ret'])
main=main.fillna(0);main

,market_ret,asset_ret
0,0.000000,0.000000
1,0.007002,0.007362
2,0.003042,0.002597
3,0.006081,0.006942
4,0.012252,0.028511
...,...,...
239,-0.006731,0.013894
240,0.000160,-0.000532
241,0.013267,0.013317
242,0.009808,0.022244


# OLS for stock beta 

In [28]:
import pandas as pd
import statsmodels.api as sm

X = main['market_ret']
y = main['asset_ret']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()


print(model.summary())
intercept, slope = model.params
print(intercept,slope)

                            OLS Regression Results                            
Dep. Variable:              asset_ret   R-squared:                       0.656
Model:                            OLS   Adj. R-squared:                  0.655
Method:                 Least Squares   F-statistic:                     462.1
Date:                Sun, 12 May 2024   Prob (F-statistic):           4.77e-58
Time:                        15:47:27   Log-Likelihood:                 820.22
No. Observations:                 244   AIC:                            -1636.
Df Residuals:                     242   BIC:                            -1629.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0008      0.001     -1.417      0.1

# Over or Undervaluation (Build the CAPM equation)

In [29]:
mu_asset=mean_historical_return(df['Bajaj Auto Ltd.']);print('actual returns',mu_asset)
y=float(market_ret);print('market returns are:',y)
x=mu_asset.item();print('asset returns are:',x)

req_ret=0.07 + slope*(y-0.07);print('required rate of return is',req_ret)

if req_ret > x:
    print('overvalued asset')
elif req_ret<x:
    print('undervalued asset')
elif req_ret == x:
    print('rightly priced')

actual returns Bajaj Auto Ltd.    1.31181
dtype: float64
market returns are: 1.5570000465494243
asset returns are: 1.311809694695004
required rate of return is 1.9362406844847666
overvalued asset


C:\Users\ashok\anaconda3\lib\site-packages\pypfopt\expected_returns.py:137: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)


In [30]:
#end